PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다. 이를 위해 pyspark 패키지를 설치한다. Py4J 패키지가 같이 설치되는데 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

# PySpark 설치

In [ ]:
!pip install pyspark==3.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Spark Session:** SparkSession은 Spark 2.0부터 엔트리 포인트로 사용된다. SparkSession을 이용해 RDD, 데이터 프레임등을 만든다. SparkSession은 SparkSession.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다

* local[*] Spark이 하나의 JVM으로 동작하고 그 안에 컴퓨터의 코어 수 만큼의 스레드가 Executor로 동작한다


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark Tutorial')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
!grep MemTotal /proc/meminfo

MemTotal:       13297200 kB


# Python <> RDD <> DataFrame

**Python 객체를 RDD로 변환해보기**

**1> Python 리스트 생성**

In [ ]:
name_list_json = [ '{"name": "keeyong"}', '{"name": "benjamin"}', '{"name": "claire"}' ]

In [ ]:
for n in name_list_json:
  print(n)

{"name": "keeyong"}
{"name": "benjamin"}
{"name": "claire"}


**2> 파이썬 리스트를 RDD로 변환**

 * RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨 (파티션)

In [ ]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [ ]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
rdd.count()

3

In [ ]:
import json

parsed_rdd = rdd.map(lambda el:json.loads(el))

In [ ]:
parsed_rdd

PythonRDD[2] at RDD at PythonRDD.scala:53

In [ ]:
parsed_rdd.collect()

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [ ]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [ ]:
parsed_name_rdd.collect()

['keeyong', 'benjamin', 'claire']

**파이썬 리스트를 데이터프레임으로 변환하기**

In [ ]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(name_list_json, StringType())

In [ ]:
df.count()

3

In [ ]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
df.select('*').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

RDD를 DataFrame으로 변환해보는 예제: 앞서 parsed_rdd를 DataFrame으로 변환해보자



In [ ]:
df_parsed_rdd = parsed_rdd.toDF()

In [ ]:
df_parsed_rdd.printSchema()

root
 |-- name: string (nullable = true)



In [ ]:
df_parsed_rdd.select('name').collect()

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]

## Spark 데이터프레임으로 로드해보기

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv

--2023-01-14 06:51:15--  https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 3.5.80.138, 3.5.80.102, 3.5.84.16, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|3.5.80.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 997 [text/csv]
Saving to: ‘name_gender.csv’

name_gender.csv     100%[===================>]     997  --.-KB/s    in 0s      

2023-01-14 06:51:16 (13.8 MB/s) - ‘name_gender.csv’ saved [997/997]



In [ ]:
df = spark.read.csv("name_gender.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [ ]:
df = spark.read.option("header", True).csv("name_gender.csv")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [ ]:
df.show()

+----------+------+
|      name|gender|
+----------+------+
|  Adaleigh|     F|
|     Amryn|Unisex|
|    Apurva|Unisex|
|    Aryion|     M|
|    Alixia|     F|
|Alyssarose|     F|
|    Arvell|     M|
|     Aibel|     M|
|   Atiyyah|     F|
|     Adlie|     F|
|    Anyely|     F|
|    Aamoni|     F|
|     Ahman|     M|
|    Arlane|     F|
|   Armoney|     F|
|   Atzhiry|     F|
| Antonette|     F|
|   Akeelah|     F|
| Abdikadir|     M|
|    Arinze|     M|
+----------+------+
only showing top 20 rows



In [ ]:
df.head(5)

[Row(name='Adaleigh', gender='F'),
 Row(name='Amryn', gender='Unisex'),
 Row(name='Apurva', gender='Unisex'),
 Row(name='Aryion', gender='M'),
 Row(name='Alixia', gender='F')]

In [ ]:
df.groupby(["gender"]).count().collect()

[Row(gender='F', count=65),
 Row(gender='M', count=28),
 Row(gender='Unisex', count=7)]

In [ ]:
df.rdd.getNumPartitions()

1

데이터프레임을 테이블뷰로 만들어서 SparkSQL로 처리해보기

In [ ]:
df.createOrReplaceTempView("namegender")

In [ ]:
namegender_group_df = spark.sql("SELECT gender, count(1) FROM namegender GROUP BY 1")

In [ ]:
namegender_group_df.collect()

[Row(gender='F', count(1)=65),
 Row(gender='M', count(1)=28),
 Row(gender='Unisex', count(1)=7)]

In [ ]:
spark.catalog.listTables()

[Table(name='namegender', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

Partition의 수 계산해보기

In [ ]:
namegender_group_df.rdd.getNumPartitions()

1

In [ ]:
two_namegender_group_df = namegender_group_df.repartition(2)

In [ ]:
two_namegender_group_df.rdd.getNumPartitions()

2